<a href="https://colab.research.google.com/github/Noor-Z1/Machine-Learning/blob/main/Viterbi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np


class HMM:
    def __init__(self, A, B, Pi):
        self.A = A
        self.B = B
        self.Pi = Pi

    def forward_log(self, O: list):
        """
        :param O: is the sequence (an array of) discrete (integer) observations, i.e. [0 , 2 ,1 ,3 , 4]
        :return: ln P(O|λ) score for the given observation, ln: natural logarithm
        """


        #instead of recursion, I follow the tabular dynamic programming approach

        num_observ = len(O)
        num_states = self.A.shape[0]

        #initializing an alpha matrix of size based on observations and number of states, where alpha is the forward variable
        alpha = np.zeros((num_observ, num_states))

        # α1(i) = P(O1 = Ok, q1 = Si) = πibim
        #initializing the 0th row of the matrix using the emission prob and initial state distribution and the given equation

        #c=0
        normalizer = np.zeros(num_observ)

        for i in range(0,num_states):
          alpha[0][i] = self.Pi[i] * self.B[i][O[0]]
          normalizer[0]+=alpha[0][i]

        #tabular/dp approach to calculate α t+1(i) (alpha value at next time step)
        #Note that to calculate the inside of the summation part of the equation, the matrix A = aij needs to be dotted with the prev alpha value

        for t in range(1, num_observ):
            normalizer[t]=0
            alpha[t, :] = alpha[t-1].dot(self.A)            #note that dotting avoids another loop
            alpha[t, :] = alpha[t, :] * self.B[:, O[t]]     #this is multiplication with bjv part of the equation. corresponding rows in B are multiplied with the corresponding alpha values using this notation


            for i in range(0,num_states):  #to find the normalizer
              normalizer[t]+=alpha[t][i]

            normalizer[t] = 1.0/normalizer[t]

            for i in range(0,num_states):  #normalizing calculated alphas.
              alpha[t][i]*= normalizer[t]


        #the return is equivalent to returning c
        #for t in range(1,num_observ):
        # c+= np.log(normalizer[t])



        return -1 * (np.sum(np.log(normalizer[1:])))


    def viterbi_log(self, O: list):


        """
        :param O: is an array of discrete (integer) observations, i.e. [0, 2, 1 ,3, 4]
        :return: the tuple (Q*, ln P(Q*|O,λ)), Q* is the most probable state sequence for the given O
        """

        #initializing tables
        num_observ = len(O)
        num_states = self.A.shape[0]

        Viterbi = np.zeros((num_observ, num_states)) #Viterbi table -> DELTA
        psi = np.zeros((num_observ, num_states))  #best path table


        #inititalization part of the 0th row using the first observation
        for i in range(0, num_states):
          Viterbi[0][i] = np.log(self.Pi[i]) + np.log(self.B[i][O[0]])    # equation 7 calculation -> need to iterate through the states to find   ln πi + ln bie



        #forward pass to get most probable prev state
        #again using iterative approach instead of recursive

        for t in range(1, num_observ):
            for j in range(num_states):
                # t is same as t+1 from the assignment equation and t-1 is same as t from assignment eq

                Viterbi[t][j]= np.max(Viterbi[t-1] + np.log(self.A[:, j])) + np.log(self.B[j][O[t]])  # equation 8 calc

                psi[t][j] = np.argmax(Viterbi[t-1] + np.log(self.A[:, j])) #equation 9 calc


        #most likely state for last time stamp, since its the last time step, I use Viterbi[-1]
        q_star = np.argmax(Viterbi[-1])


        #backward tracing to get Max Likelihood seq (as described in assignment)
        QSTAR_seq=[]
        QSTAR_seq.append(int(q_star))
        #backward tracing to get Max Likelihood seq (as described in assignment)
        t= num_observ -1


        while(t>0):
            QSTAR_seq.append(int(psi[t][int(QSTAR_seq[-1])]))  #need to convert the last element of qstar to int so it works as index
            t = t-1


        QSTAR_seq = QSTAR_seq[::-1]   #need to reverse the qstar list to get the correct order since in back tracking the last is entered first in the array



        return (QSTAR_seq, np.max(Viterbi[-1]))


In [ ]:
#from HMM import HMM


import numpy as np

A = np.array([[0.4, 0.6],
              [0.7, 0.3]], dtype=np.float128)

B = np.array([[0.3, 0.4, 0.3],
              [0.1, 0.2, 0.7]], dtype=np.float128)

Pi = np.array([0.6, 0.4], dtype=np.float128)

hmm = HMM(A, B, Pi)

O1 = [2, 1, 0]
O2 = [0,0,2,1,0]
print(hmm.forward_log(O1))
print(hmm.forward_log(O2))


print(hmm.viterbi_log(O1))
print(hmm.viterbi_log(O2))



-3.5574302276937386
-6.620857514019081
([1, 0, 0], -4.666194887825866)
([0, 0, 1, 0, 0], -8.09579174400972)
